<span style="color:darkblue"><font size="5"> DeCockHousePrice Dataset: SalePrice Prediction</font></span> 
    
    SVR vs Linear Regression

In [587]:
import pandas as pd
import seaborn as sns; sns.set(color_codes=True)
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
import warnings; warnings.simplefilter('ignore')
import numpy as np
np.random.seed(10)
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline

In [588]:
df=pd.read_csv('data/train.csv')

# Data cleaning

In [589]:
cg_cols = ['Alley', 'PoolQC', 'MiscFeature', 'Fence', 'FireplaceQu', 'GarageType',
    'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond',
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType']
dc_cols = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath',
    'BsmtHalfBath', 'GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea']
ot_cols = ['Electrical', 'Exterior1st', 'Exterior2nd', 'Functional', 'KitchenQual',
    'SaleType', 'Utilities']
for col in dc_cols:
    df[col].replace(np.nan, 0, inplace=True)

for col in cg_cols:
    df[col].replace(np.nan, 'None', inplace=True)

for col in ot_cols:
    df[col].replace(np.nan, df[col].mode()[0], inplace=True)
    
# Filling MSZoning according to MSSubClass
df['MSZoning'] = df.groupby('MSSubClass')['MSZoning'].apply(
    lambda x: x.fillna(x.mode()[0]))

# Filling LotFrontage according to Neighborhood

df['LotFrontage']=df.groupby(['Neighborhood'])['LotFrontage'].apply(lambda x: x.fillna(x.median()))

In [590]:
df.drop('Id',axis=1,inplace=True)

#binning with rare values
col_rare = ['Condition1', 'Condition2', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
    'Heating', 'Electrical', 'Functional', 'SaleType']

for col in col_rare:
    mask = df[col].isin(
        df[col].value_counts()[df[col].value_counts() < 10].index)
    df[col][mask] = 'Other'

# Features which numerical on data but should be treated as category.
df['MSSubClass'] = df['MSSubClass'].astype(str)

# cyclical feature transformation
from math import pi
df['MoSold']=2*pi*df['MoSold']/df['MoSold'].max()
df["cos_MoSold"] = np.cos(df["MoSold"])
df["sin_MoSold"] = np.sin(df["MoSold"])

In [570]:
#log transform skewed numeric features:

from scipy.stats import skew
numeric_feats = df.dtypes[df.dtypes != "object"].index

skewed_feats = df[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

df[skewed_feats] = np.log1p(df[skewed_feats])

In [591]:
neigh_map={'MeadowV': 1,'IDOTRR': 1,
    'BrDale': 1, 'BrkSide': 2,
    'OldTown': 2, 'Edwards': 2,
    'Sawyer': 3,    'Blueste': 3,
    'SWISU': 3,    'NPkVill': 3,
    'NAmes': 3,    'Mitchel': 4,
    'SawyerW': 5,    'NWAmes': 5,
    'Gilbert': 5,    'Blmngtn': 5,
    'CollgCr': 5,    'ClearCr': 6,
    'Crawfor': 6,    'Veenker': 7,
    'Somerst': 7,    'Timber': 8,
    'StoneBr': 9,    'NridgHt': 10,
    'NoRidge': 10}
    
df['Neighborhood'] = df['Neighborhood'].map(neigh_map).astype('int')

ext_map = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df['ExterCond'] = df['ExterCond'].map(ext_map).astype('int')
df['ExterQual'] = df['ExterQual'].map(ext_map).astype('int')

bsm_map = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df['BsmtQual'] = df['BsmtQual'].map(bsm_map).astype('int')
df['BsmtCond'] = df['BsmtCond'].map(bsm_map).astype('int')
bsmf_map = {'None': 0, 'Unf': 1,'LwQ': 2,'Rec': 3, 'BLQ': 4, 'ALQ': 5,'GLQ': 6}

df['BsmtFinType1'] = df['BsmtFinType1'].map(bsmf_map).astype('int')
df['BsmtFinType2'] = df['BsmtFinType2'].map(bsmf_map).astype('int')
heat_map = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df['HeatingQC'] = df['HeatingQC'].map(heat_map).astype('int')
df['KitchenQual'] = df['KitchenQual'].map(heat_map).astype('int')
df['FireplaceQu'] = df['FireplaceQu'].map(bsm_map).astype('int')
df['GarageCond'] = df['GarageCond'].map(bsm_map).astype('int')
df['GarageQual'] = df['GarageQual'].map(bsm_map).astype('int')

# Getting dummy variables for nominal categorical features
df = pd.get_dummies(data=df)

In [592]:
df.head()

,LotFrontage,LotArea,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_COD,SaleType_New,SaleType_Other,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,65.0,8450,5,7,5,2003,2003,196.0,4,3,...,0,0,0,1,0,0,0,0,1,0
1,80.0,9600,7,6,8,1976,1976,0.0,3,3,...,0,0,0,1,0,0,0,0,1,0
2,68.0,11250,5,7,5,2001,2002,162.0,4,3,...,0,0,0,1,0,0,0,0,1,0
3,60.0,9550,6,7,5,1915,1970,0.0,3,3,...,0,0,0,1,1,0,0,0,0,0
4,84.0,14260,10,8,5,2000,2000,350.0,4,3,...,0,0,0,1,0,0,0,0,1,0


# Feature Selection
    
     Based on Correlation and P-value 

In [593]:
corr = df.corr()
#Remove one of two features that have a correlation higher than 0.9
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
selected_columns = df.columns[columns]

df = df[selected_columns]

In [594]:
import statsmodels.regression.linear_model as sm
X=df.drop('SalePrice',axis=1).values
y=df.loc[:,'SalePrice'].values

selected_columns

selected_columns=selected_columns.tolist()

selected_columns.remove('SalePrice')

selected_columns=np.array(selected_columns)

numVars = len(selected_columns)-1
for i in range(0, numVars):
    regressor_OLS = sm.OLS(y, X).fit()
    maxVar = max(regressor_OLS.pvalues).astype(float)
    if maxVar > 0.05:
        for j in range(0, numVars - i):
            if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                X = np.delete(X, j, 1)
                selected_columns = np.delete(selected_columns, j)

selected_columns.size

119

# Data Scaling and transformation

In [595]:
y=np.log(y)
y

array([12.24769432, 12.10901093, 12.31716669, ..., 12.49312952,
       11.86446223, 11.90158345])

In [596]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
X = minmax.fit_transform(X)

In [598]:
from sklearn.decomposition import PCA
pca = PCA(n_components=60)
X = pca.fit_transform(X)
pca.explained_variance_ratio_.sum()

0.9707082029962816

# Model training and evaluation
    SVR versus Ridge

In [599]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
X_train_val.shape

(1168, 60)

In [521]:
from sklearn.svm import SVR

from sklearn.model_selection import cross_val_score,GridSearchCV

In [522]:
param_grid = [
  {'C': [0.01,0.1,1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [0.01,0.1,1, 10, 100, 1000], 'gamma': [100,10,1,0.1,0.01,0.001, 0.001], \
   'kernel': ['rbf']},]
grid_search=GridSearchCV(SVR(),param_grid,cv=5)
grid_search.fit(X_train_val,y_train_val)
grid_search.best_params_

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [523]:
grid_search.best_score_

0.8740967615493374

In [524]:
grid_search.score(X_test,y_test)

0.8227719769328563

In [525]:
y_pred=grid_search.predict(X_test)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test, y_pred))

0.16380566128834084

In [528]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
for alpha in [0.0001,0.001,0.01,0.1,1,10,100]:
    model = Ridge(alpha=alpha)
    print(f'alpha= {alpha}')
    print(f'Average R2_score:{ cross_val_score(model,X_train_val, y_train_val,cv=3).mean()}')
    print('\n')

alpha= 0.0001
Average R2_score:0.8641821784609308


alpha= 0.001
Average R2_score:0.8641834498060416


alpha= 0.01
Average R2_score:0.8641961175252617


alpha= 0.1
Average R2_score:0.8643182969380492


alpha= 1
Average R2_score:0.8651542752864443


alpha= 10
Average R2_score:0.8586766711174278


alpha= 100
Average R2_score:0.7414474467595763




In [530]:
ridge_model=Ridge(alpha=10)
ridge_model.fit(X_train_val, y_train_val)
cross_val_score(ridge_model,X_train_val, y_train_val,cv=5)

array([0.88259859, 0.8280492 , 0.85471439, 0.88222856, 0.8735736 ])

In [531]:
ridge_model.score(X_test,y_test)

0.8241236833260553

In [532]:
y_pred=ridge_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

0.1631797981856338

# Bias and Variance Analysis and Conclusions

In [534]:
def svr_bias_variance(X,y):
    from mlxtend.evaluate import bias_variance_decomp
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=123,
                                                    shuffle=True)
    model = SVR(C=10, gamma=0.01, kernel='rbf')

    avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        model, X_train, y_train, X_test, y_test, 
        loss='mse',
        random_seed=123)

    print('Average expected loss: %.3f' % avg_expected_loss)
    print('Average bias: %.3f' % avg_bias)
    print('Average variance: %.3f' % avg_var)

In [535]:
def ridge_bias_variance(X,y):
    from mlxtend.evaluate import bias_variance_decomp
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=123,
                                                    shuffle=True)
    model = Ridge(alpha=10)

    avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        model, X_train, y_train, X_test, y_test, 
        loss='mse',
        random_seed=123)

    print('Average expected loss: %.3f' % avg_expected_loss)
    print('Average bias: %.3f' % avg_bias)
    print('Average variance: %.3f' % avg_var)

In [539]:
print('Bias and Variance of SVR model:')
svr_bias_variance(X,y)
print('\n Bias and Variance of Ridge model:')
ridge_bias_variance(X,y)

Bias and Variance of SVR model:
Average expected loss: 0.378
Average bias: 0.354
Average variance: 0.023

 Bias and Variance of Ridge model:
Average expected loss: 0.378
Average bias: 0.362
Average variance: 0.016


In [533]:
print(f'Performance of SVM on unseen data: \n    R2 score: {grid_search.score(X_test,y_test)}\n' )
print(f'Performance of Ridge on unseen data: \n   R2 score: {ridge_model.score(X_test,y_test)}' )

Performance of SVM on unseen data: 
    R2 score: 0.8227719769328563

Performance of Ridge on unseen data: 
   R2 score: 0.8241236833260553



- **SVM model's performance on unseen data is  similar as Ridge**
- **Bias is the problem**

# solution
- Collect more valuable features
- Manully creat new features from existing features
- Change feature selection strategy

In [559]:
X=df.drop('SalePrice',axis=1).values
y=df.loc[:,'SalePrice'].values

In [562]:
y[0]

12.24769911637256

In [600]:
from pyearth import Earth

mars = Earth(max_degree=1, penalty=1.0, endspan=5)

In [601]:
# gbrmars=AdaBoostRegressor(base_estimator=mars,)

cross_val_score(mars,X,y,cv=5).mean()

# boosted_model = AdaBoostRegressor(base_estimator=mars, n_estimators=25,
#                                   learning_rate=0.1, loss="exponential")

0.8161191027788333

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_actual, y_predicted))

In [544]:
from sklearn.ensemble import AdaBoostRegressor
boosted_model = AdaBoostRegressor(base_estimator=mars, n_estimators=25,
                                  learning_rate=0.1, loss="exponential")

In [602]:
cross_val_score(boosted_model,X,y,cv=3).mean()


0.7875485427633672